## Scoring POC Notebook

In [1]:
import java.nio.file.Paths
import org.apache.spark.sql.Row
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import scala.collection.JavaConversions.mapAsScalaMap
import scala.collection.JavaConversions._
import org.apache.spark.sql.expressions.Window
import org.yaml.snakeyaml.Yaml
import scala.io.Source

In [2]:
val input_dir = Paths.get(System.getProperty("user.home"), "data", "ot", "extract")
val output_dir = Paths.get(System.getProperty("user.home"), "data", "ot", "results")
val input_file = input_dir.resolve("evidence.json")
val df = spark.read.json(input_file.toString())

input_dir = /home/eczech/data/ot/extract
output_dir = /home/eczech/data/ot/results
input_file = /home/eczech/data/ot/extract/evidence.json
df = [access_level: string, disease: struct<biosample: struct<id: string, name: string>, efo_info: struct<efo_id: string, label: string ... 2 more fields> ... 3 more fields> ... 12 more fields]


[access_level: string, disease: struct<biosample: struct<id: string, name: string>, efo_info: struct<efo_id: string, label: string ... 2 more fields> ... 3 more fields> ... 12 more fields]

In [52]:
// df.printSchema()

Name: Syntax Error.
Message: 
StackTrace: 

In [53]:
df.count()

1626544

In [73]:
// df.printSchema

Name: Syntax Error.
Message: 
StackTrace: 

In [78]:
val evidence_fields = List(
    "evidence.resource_score.value",
    "evidence.drug2clinic.resource_score.value",
    "evidence.target2drug.resource_score.value",
    "evidence.log2_fold_change.value",
    "evidence.log2_fold_change.percentile_rank",
    "unique_association_fields.cases",
    "evidence.variant2disease.resource_score.value",
    "evidence.gene2variant"
)
val evidence_cols = evidence_fields.map(c => col(c).alias(c.replace(".", "_")))

val dfe = df
    .select(
        $"target.id".as("target_id"),
        $"type".as("type_id"),
        $"private.efo_codes".as("efo_codes"),
        $"disease.id".as("disease_id"),
        $"scores.association_score".as("score"),
        $"sourceID".as("source_id"),
        $"id",
        struct(evidence_cols:_*).as("resource_data")
    )

dfe.select("resource_data.evidence_log2_fold_change_value").show(3)

+-------------------------------+
|evidence_log2_fold_change_value|
+-------------------------------+
|                            2.1|
|                            3.8|
|                            3.5|
+-------------------------------+
only showing top 3 rows



evidence_fields = List(evidence.resource_score.value, evidence.drug2clinic.resource_score.value, evidence.target2drug.resource_score.value, evidence.log2_fold_change.value, evidence.log2_fold_change.percentile_rank, unique_association_fields.cases, evidence.variant2disease.resource_score.value, evidence.gene2variant)
evidence_cols = List(evidence.resource_score.value AS `evidence_resource_score_value`, evidence.drug2clinic.resource_score.value AS `evidence_drug2clinic_resource_score_value`, evidence.target2drug.resource_score.value AS `evidence_target2drug_resource_score_value`, evidence.log2_fold_change.value AS `evidence_log2_fold_change_value`, evidence.log2_fold_change.percentile_rank AS `evidence_log2_fold_change_percentile_rank`, un...


lastException: Throwable = null


List(evidence.resource_score.value AS `evidence_resource_score_value`, evidence.drug2clinic.resource_score.value AS `evidence_drug2clinic_resource_score_value`, evidence.target2drug.resource_score.value AS `evidence_target2drug_resource_score_value`, evidence.log2_fold_change.value AS `evidence_log2_fold_change_value`, evidence.log2_fold_change.percentile_rank AS `evidence_log2_fold_change_percentile_rank`, un...

In [69]:
// val dfe = df.select(
//         $"target.id".as("target_id"),
//         $"private.efo_codes".as("efo_codes"),
//         $"disease.id".as("disease_id"),
//         $"scores.association_score".as("score"),
//         $"sourceID".as("source_id"),
//         $"evidence".as("resource_data"),
//         $"id"
//     )

Name: Syntax Error.
Message: 
StackTrace: 

In [70]:
dfe.show(5, 18)

+---------------+--------------+------------------+-----------+------------------+----------------+------------------+------------------+
|      target_id|       type_id|         efo_codes| disease_id|             score|       source_id|                id|     resource_data|
+---------------+--------------+------------------+-----------+------------------+----------------+------------------+------------------+
|ENSG00000134438|rna_expression|[EFO_0000384, E...|EFO_0000384|0.1130747022339...|expression_atlas|38c0851b3dba497...|[3.64E-6,,, 2.1...|
|ENSG00000162747|rna_expression|[EFO_0000384, E...|EFO_0000384|            0.3762|expression_atlas|899d05b84c11212...|[6.28E-35,,, 3....|
|ENSG00000105989|rna_expression|[EFO_0000384, E...|EFO_0000384|            0.3465|expression_atlas|a56ee52c9af9d70...|[1.53E-20,,, 3....|
|ENSG00000261520|rna_expression|[EFO_0000384, E...|EFO_0000384|0.3266999999999...|expression_atlas|b3c46e0faca5bb8...|[7.26E-17,,, 3....|
|ENSG00000255398|rna_expression|[E

In [63]:
dfe.groupBy("type").count().orderBy("count").show()

+-------------------+------+
|               type| count|
+-------------------+------+
|   somatic_mutation| 70554|
|   affected_pathway| 87104|
|     rna_expression|204229|
|genetic_association|380852|
|         known_drug|383122|
|       animal_model|500683|
+-------------------+------+



In [85]:
dfe.groupBy("source_id", "type_id").count().orderBy("count").show()

+------------------+-------------------+------+
|         source_id|            type_id| count|
+------------------+-------------------+------+
|   uniprot_somatic|   somatic_mutation|   284|
|           progeny|   affected_pathway|   308|
|            sysbio|   affected_pathway|   408|
|    gene2phenotype|genetic_association|  1586|
|            crispr|   affected_pathway|  1659|
|           intogen|   somatic_mutation|  2375|
|uniprot_literature|genetic_association|  4553|
|       eva_somatic|   somatic_mutation|  7585|
|          reactome|   affected_pathway| 10159|
|  genomics_england|genetic_association| 10528|
|           uniprot|genetic_association| 30480|
|    phewas_catalog|genetic_association| 55987|
|cancer_gene_census|   somatic_mutation| 60310|
|        slapenrich|   affected_pathway| 74570|
|               eva|genetic_association| 96734|
|      gwas_catalog|genetic_association|180984|
|  expression_atlas|     rna_expression|204229|
|            chembl|         known_drug|

In [31]:
dfe.groupBy("source_id").count().orderBy("count").show()

+------------------+------+
|         source_id| count|
+------------------+------+
|   uniprot_somatic|   284|
|           progeny|   308|
|            sysbio|   408|
|    gene2phenotype|  1586|
|            crispr|  1659|
|           intogen|  2375|
|uniprot_literature|  4553|
|       eva_somatic|  7585|
|          reactome| 10159|
|  genomics_england| 10528|
|           uniprot| 30480|
|    phewas_catalog| 55987|
|cancer_gene_census| 60310|
|        slapenrich| 74570|
|               eva| 96734|
|      gwas_catalog|180984|
|  expression_atlas|204229|
|            chembl|383122|
|         phenodigm|500683|
+------------------+------+



In [8]:
dfe.select("efo_codes").show(3, 100)

+------------------------------------------------------------------------------+
|                                                                     efo_codes|
+------------------------------------------------------------------------------+
|[EFO_0000384, EFO_0000405, EFO_0000540, EFO_0000408, EFO_0003767, EFO_0005140]|
|[EFO_0000384, EFO_0000405, EFO_0000540, EFO_0000408, EFO_0003767, EFO_0005140]|
|[EFO_0000384, EFO_0000405, EFO_0000540, EFO_0000408, EFO_0003767, EFO_0005140]|
+------------------------------------------------------------------------------+
only showing top 3 rows



In [9]:
dfe.select(size($"efo_codes").as("n")).groupBy("n").count().orderBy("n").show()

+---+------+
|  n| count|
+---+------+
|  1|     4|
|  2| 31382|
|  3|117817|
|  4|177100|
|  5|143127|
|  6|199967|
|  7| 99531|
|  8|105265|
|  9|129871|
| 10|155527|
| 11|104358|
| 12| 57374|
| 13| 56029|
| 14| 30258|
| 15| 28318|
| 16| 13342|
| 17| 17993|
| 18| 12379|
| 19| 21961|
| 20| 18028|
+---+------+
only showing top 20 rows



In [63]:
// Verify that the efo_codes list contains the disease_id before exploding based on 
// the efo_codes field (which could result in rows being lost if efo_codes is empty
// or does not contain the primary disease id)
dfe.map(r => r.getAs[Seq[String]]("efo_codes") contains r.getAs[String]("disease_id"))
    .groupBy("value").count().show()

+-----+-------+
|value|  count|
+-----+-------+
| true|1626544|
+-----+-------+



#### Prepare Disease Ids

In [79]:
val direct_data_sources = List("expression_atlas")
val dfi = dfe
    .withColumn("is_direct_source", $"source_id".isin(direct_data_sources:_*))
    .withColumn(
        "efo_ids", 
        when($"is_direct_source", array($"disease_id"))
        .otherwise($"efo_codes")
    )
    .withColumnRenamed("disease_id", "orig_disease_id")
    
dfi

direct_data_sources = List(expression_atlas)
dfi = [target_id: string, type_id: string ... 8 more fields]


[target_id: string, type_id: string ... 8 more fields]

In [80]:
dfi.printSchema()

root
 |-- target_id: string (nullable = true)
 |-- type_id: string (nullable = true)
 |-- efo_codes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- orig_disease_id: string (nullable = true)
 |-- score: double (nullable = true)
 |-- source_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- resource_data: struct (nullable = false)
 |    |-- evidence_resource_score_value: double (nullable = true)
 |    |-- evidence_drug2clinic_resource_score_value: double (nullable = true)
 |    |-- evidence_target2drug_resource_score_value: long (nullable = true)
 |    |-- evidence_log2_fold_change_value: double (nullable = true)
 |    |-- evidence_log2_fold_change_percentile_rank: long (nullable = true)
 |    |-- unique_association_fields_cases: string (nullable = true)
 |    |-- evidence_variant2disease_resource_score_value: double (nullable = true)
 |    |-- evidence_gene2variant: struct (nullable = true)
 |    |    |-- date_asserted: string (nullable = t

In [82]:
dfi.select($"resource_data.evidence_gene2variant".isNull.as("is_null")).groupBy("is_null").count().show()

+-------+-------+
|is_null|  count|
+-------+-------+
|   true|1262359|
|  false| 364185|
+-------+-------+



lastException: Throwable = null


#### Resource Scores

In [84]:
val scoringUdf = udf {
    (typeId: String, sourceId: String, resourceData: Row) => {
        var evidence = resourceData.getAs[Row]("evidence_gene2variant")
        evidence == null
    }
}
val dfir = dfi
    .withColumn("score_resource", scoringUdf($"type_id", $"source_id", $"resource_data"))
    .groupBy("score_resource").count().show()

+--------------+-------+
|score_resource|  count|
+--------------+-------+
|          true|1262359|
|         false| 364185|
+--------------+-------+



scoringUdf = UserDefinedFunction(<function3>,BooleanType,None)


dfir: Unit = ()


UserDefinedFunction(<function3>,BooleanType,None)

In [67]:
val scoringUdf = udf {
    (typeId: String, sourceId: String, resourceData: Row) => typeId + ":" + sourceId
}
val dfir = dfi.withColumn("score_resource", scoringUdf($"type_id", $"source_id", $"resource_data"))
dfir.show(3)

+---------------+--------------+--------------------+---------------+-------------------+----------------+--------------------+--------------------+----------------+-------------+--------------------+
|      target_id|          type|           efo_codes|orig_disease_id|              score|       source_id|                  id|       resource_data|is_direct_source|      efo_ids|      score_resource|
+---------------+--------------+--------------------+---------------+-------------------+----------------+--------------------+--------------------+----------------+-------------+--------------------+
|ENSG00000134438|rna_expression|[EFO_0000384, EFO...|    EFO_0000384|0.11307470223393612|expression_atlas|38c0851b3dba497d7...|[3.64E-6,,, 2.1, ...|            true|[EFO_0000384]|rna_expression:ex...|
|ENSG00000162747|rna_expression|[EFO_0000384, EFO...|    EFO_0000384|             0.3762|expression_atlas|899d05b84c11212ad...|[6.28E-35,,, 3.8,...|            true|[EFO_0000384]|rna_expression:ex

scoringUdf = UserDefinedFunction(<function3>,StringType,None)
dfir = [target_id: string, type: string ... 9 more fields]


[target_id: string, type: string ... 9 more fields]

#### Explode By Disease

In [56]:
val dfie = dfir
    .select(
        $"id", $"source_id", $"orig_disease_id", $"target_id", $"resource_data", $"score",
        explode($"efo_ids").as("disease_id")
    )
    .withColumn(
        "is_direct_id",
        $"orig_disease_id" === $"disease_id"
    )
dfie

dfie = [id: string, source_id: string ... 6 more fields]


[id: string, source_id: string ... 6 more fields]

In [57]:
dfie.count()

14067077

In [58]:
dfie.show(3)

+--------------------+----------------+---------------+---------------+--------------------+-------------------+-----------+------------+
|                  id|       source_id|orig_disease_id|      target_id|       resource_data|              score| disease_id|is_direct_id|
+--------------------+----------------+---------------+---------------+--------------------+-------------------+-----------+------------+
|38c0851b3dba497d7...|expression_atlas|    EFO_0000384|ENSG00000134438|[3.64E-6,,, 2.1, ...|0.11307470223393612|EFO_0000384|        true|
|899d05b84c11212ad...|expression_atlas|    EFO_0000384|ENSG00000162747|[6.28E-35,,, 3.8,...|             0.3762|EFO_0000384|        true|
|a56ee52c9af9d7015...|expression_atlas|    EFO_0000384|ENSG00000105989|[1.53E-20,,, 3.5,...|             0.3465|EFO_0000384|        true|
+--------------------+----------------+---------------+---------------+--------------------+-------------------+-----------+------------+
only showing top 3 rows



In [13]:
dfie.filter(
    $"disease.id" === "EFO_0001645" &&
    $"target.id" === "ENSG00000226777" &&
    $"sourceID" === "sysbio"
).show()

+--------------------+---------+---------------+---------------+-----+-----------+------------+
|                  id|source_id|orig_disease_id|      target_id|score| disease_id|is_direct_id|
+--------------------+---------+---------------+---------------+-----+-----------+------------+
|abb040109d9414a1f...|   sysbio|    EFO_0001645|ENSG00000226777|0.859|EFO_0003777|       false|
|abb040109d9414a1f...|   sysbio|    EFO_0001645|ENSG00000226777|0.859|EFO_0000408|       false|
|abb040109d9414a1f...|   sysbio|    EFO_0001645|ENSG00000226777|0.859|EFO_0000319|       false|
|abb040109d9414a1f...|   sysbio|    EFO_0001645|ENSG00000226777|0.859|EFO_0001645|        true|
+--------------------+---------+---------------+---------------+-----+-----------+------------+



In [ ]:
(
    for (v <- Seq(0, 1)) 
    yield {dfie.filter($"is_direct_id" === v).limit(3)}
) reduce(_ union _) show(6, 12)

In [ ]:
dfie.groupBy("source_id").pivot("is_direct_id").agg(count("id")).show()

In [ ]:
dfie.groupBy("target_id", "disease_id").count().withColumnRenamed("count", "n")
    .groupBy("n").count().orderBy($"n".desc).show

In [ ]:
import java.util.{Map => JMap}
def using[A](r: Source)(f: Source => A): A = {try f(r) finally r.close()}
def loadConfig(path: String): JMap[String, Any] = {
    val content = using(Source.fromFile(path))(f => f.mkString)
    (new Yaml).load(content).asInstanceOf[JMap[String, Any]]
}
def loadScoringConfig(path: String): Map[String, Double] = {
    mapAsScalaMap(
        loadConfig(path)
        .get("scoring_weights").asInstanceOf[JMap[String, Any]]
        .get("source").asInstanceOf[JMap[String, Double]]
    ).toMap
}
val config = loadScoringConfig("../config/scoring.yml")
config

In [ ]:
val lkp = typedLit(config)
val dfse = dfie
    .withColumnRenamed("score", "score_evidence")
    .withColumn("score_source", $"score_evidence" * coalesce(lkp($"source_id"), lit(1.0)))
dfse.show(3)

In [177]:
var w = Window.partitionBy("target_id", "disease_id", "source_id").orderBy($"score_source".desc)
val dfseh = dfse
    .withColumn("rid", row_number().over(w))
    .withColumn("score", $"score_source" / pow($"rid", 2.0))
dfseh.show(5)

+--------------------+----------------+---------------+---------------+--------------+--------------+------------+------------+---+-------+
|                  id|       source_id|orig_disease_id|      target_id|score_evidence|    disease_id|is_direct_id|score_source|rid|  score|
+--------------------+----------------+---------------+---------------+--------------+--------------+------------+------------+---+-------+
|c2ddf8443100ed7f1...|       phenodigm|  Orphanet_3337|ENSG00000000460|        0.9317|   EFO_0009566|       false|     0.18634|  1|0.18634|
|c2ddf8443100ed7f1...|       phenodigm|  Orphanet_3337|ENSG00000000460|        0.9317| Orphanet_3337|        true|     0.18634|  1|0.18634|
|5f0d033bca543b007...|genomics_england| Orphanet_33574|ENSG00000001084|           1.0|Orphanet_68367|       false|         1.0|  1|    1.0|
|a622181d1a443117e...|genomics_england| Orphanet_33574|ENSG00000001084|           1.0|Orphanet_68367|       false|         1.0|  2|   0.25|
|6e6fcb2a7b6ef8285..

w = org.apache.spark.sql.expressions.WindowSpec@35ae1cc2
dfseh = [id: string, source_id: string ... 8 more fields]


[id: string, source_id: string ... 8 more fields]

In [178]:
dfseh.describe("score").show()

+-------+--------------------+
|summary|               score|
+-------+--------------------+
|  count|            14067077|
|   mean| 0.05697707589949231|
| stddev|  0.1466786322141555|
|    min|4.590854684798087...|
|    max|                 1.0|
+-------+--------------------+



In [186]:
// Aggregate association to source level
// - clip summed scores at 1
// - take max is_direct before applying max again at association level
// - See: https://github.com/opentargets/data_pipeline/blob/7098546ee09ca1fc3c690a0bd6999b865ddfe646/mrtarget/modules/Association.py#L285
//   for how scores are clipped at source level but not at datatype or overall levels
val dfses = dfseh.groupBy("target_id", "disease_id", "source_id")
    .agg(sum("score").as("score_raw"), max("is_direct_id").as("is_direct"))
    .withColumn("score", when($"score_raw" > 1, 1).otherwise($"score_raw"))
dfses.show(3)

+---------------+--------------+----------------+---------+---------+-------+
|      target_id|    disease_id|       source_id|score_raw|is_direct|  score|
+---------------+--------------+----------------+---------+---------+-------+
|ENSG00000000460|   EFO_0009566|       phenodigm|  0.18634|    false|0.18634|
|ENSG00000000460| Orphanet_3337|       phenodigm|  0.18634|     true|0.18634|
|ENSG00000001084|Orphanet_68367|genomics_england|     1.25|    false|    1.0|
+---------------+--------------+----------------+---------+---------+-------+
only showing top 3 rows



dfses = [target_id: string, disease_id: string ... 4 more fields]


[target_id: string, disease_id: string ... 4 more fields]

In [187]:
import org.apache.spark.ml.feature.Bucketizer
val bucketizer = new Bucketizer()
    .setInputCol("score_raw")
    .setOutputCol("score_bin")
    .setSplits(Array(0.0, 0.1, 0.5, 0.8, 1.0, 1.5, 2.0, 1000.0))
bucketizer.transform(dfses).groupBy("score_bin").count().show()

+---------+-------+
|score_bin|  count|
+---------+-------+
|      0.0| 339977|
|      1.0|1474378|
|      4.0| 181197|
|      3.0|  66604|
|      2.0| 212497|
|      5.0|  16993|
+---------+-------+



bucketizer = bucketizer_35fb4623cc5e


bucketizer_35fb4623cc5e

In [188]:
dfses.describe("score_raw").show()

+-------+--------------------+
|summary|           score_raw|
+-------+--------------------+
|  count|             2291646|
|   mean|  0.3497490074440019|
| stddev|  0.3227700291513107|
|    min|2.124299626275006...|
|    max|   1.644636205517087|
+-------+--------------------+



In [189]:
// Aggregate to association level
var w = Window.partitionBy("target_id", "disease_id").orderBy($"score".desc)
val dfsea = dfses
    .withColumn("rid", row_number().over(w))
    .withColumn("score", $"score" / pow($"rid", 2.0))
    .groupBy("target_id", "disease_id")
    .agg(    
        sum("score").as("score"), 
        max("is_direct").as("is_direct"),
        collect_set("source_id").as("source_ids")
    )
dfsea.show(5)

+---------------+--------------+-------------------+---------+-----------+
|      target_id|    disease_id|              score|is_direct| source_ids|
+---------------+--------------+-------------------+---------+-----------+
|ENSG00000000938|Orphanet_44890|                0.2|     true|   [chembl]|
|ENSG00000000971| Orphanet_1505|0.23495000000000002|    false|[phenodigm]|
|ENSG00000000971|Orphanet_98687|                0.2|    false|[phenodigm]|
|ENSG00000001497|   EFO_0000178|           0.405682|     true|   [crispr]|
|ENSG00000001626| Orphanet_2924|            0.22535|     true|[phenodigm]|
+---------------+--------------+-------------------+---------+-----------+
only showing top 5 rows



w = org.apache.spark.sql.expressions.WindowSpec@3e8739d6
dfsea = [target_id: string, disease_id: string ... 3 more fields]


[target_id: string, disease_id: string ... 3 more fields]

In [196]:
var path = output_dir.resolve("score_source.parquet")
dfses.write.format("parquet").mode("overwrite").save(path.toString())
path

path = /home/eczech/data/ot/results/score_source.parquet


/home/eczech/data/ot/results/score_source.parquet

In [197]:
var path = output_dir.resolve("score_association.parquet")
dfsea.write.format("parquet").mode("overwrite").save(path.toString())
path

path = /home/eczech/data/ot/results/score_association.parquet


/home/eczech/data/ot/results/score_association.parquet

In [201]:
def testtr(df: DataFrame): DataFrame = {
    df
}
dfsea.transform(testtr).show(1)

testtr: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


+---------------+--------------+-----+---------+----------+
|      target_id|    disease_id|score|is_direct|source_ids|
+---------------+--------------+-----+---------+----------+
|ENSG00000000938|Orphanet_44890|  0.2|     true|  [chembl]|
+---------------+--------------+-----+---------+----------+
only showing top 1 row

